In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, ToolMessage
from tavily import TavilyClient

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# --- Define Tools ---
# This time, we'll give our agent two tools to choose from.
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def search_tavily(query: str) -> str:
    """Searches the web for the user's query using the Tavily API."""
    tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    results = tavily.search(query=query, max_results=2)
    return str(results['results'])

tools = [multiply, search_tavily]

# --- Define State and Nodes ---
# The state will now just be a list of messages to keep track of the conversation.
class GraphState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

def call_llm(state: GraphState):
    # The primary node that calls the LLM. It can either generate a
    # direct response or decide to call one or more tools.
    llm = ChatOpenAI(model="gpt-4o")
    messages = state['messages']
    
    # We bind our tools to the LLM so it knows about them.
    llm_with_tools = llm.bind_tools(tools)
    
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tool(state: GraphState):
    # This node executes the tools that the LLM decided to call.
    last_message = state['messages'][-1]
    tool_messages = []
    
    # Loop through all the tool calls in the last message
    for tool_call in last_message.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        # Find the correct tool to call
        if tool_name == "multiply":
            result = multiply.invoke(tool_args)
        elif tool_name == "search_tavily":
            result = search_tavily.invoke(tool_args)
        
        # Append the tool's result to our list of messages
        tool_messages.append(ToolMessage(content=str(result), tool_call_id=tool_call['id']))
        
    return {"messages": tool_messages}

# --- Define the Router (Conditional Edge) ---
def should_call_tool(state: GraphState):
    # Our router checks the last message to see if it contains tool calls.
    last_message = state['messages'][-1]
    if last_message.tool_calls:
        # If it does, we route to the 'call_tool' node.
        return "call_tool"
    # Otherwise, the graph can end.
    return END

# --- Build the Graph ---
workflow = StateGraph(GraphState)

# Add the nodes
workflow.add_node("llm", call_llm)
workflow.add_node("call_tool", call_tool)

# The entry point is the 'llm' node.
workflow.set_entry_point("llm")

# Add the conditional edge for routing
workflow.add_conditional_edges(
    "llm",
    should_call_tool,
)

# After calling a tool, we always loop back to the LLM
workflow.add_edge("call_tool", "llm")

# Compile the graph
app = workflow.compile()

# --- Run the Agent ---
print("--- Running Agent with a tool-calling question ---")
# This input should trigger the 'search_tavily' tool.
tool_input = {"messages": [("user", "What's the weather like in San Francisco?")]}
for event in app.stream(tool_input):
    for value in event.values():
        print(value)

print("\n--- Running Agent with a direct question ---")
# This input should be answered directly by the LLM without using a tool.
direct_input = {"messages": [("user", "What is 2 + 2?")]}
for event in app.stream(direct_input):
    for value in event.values():
        print(value)

--- Running Agent with a tool-calling question ---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VMvCRxAUxacA29DKlpwSdmYr', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'search_tavily'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 81, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f64f290af2', 'id': 'chatcmpl-CRHG3twXhAzGDfN8KIIAiYODZomuR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8ee33221-ec97-4adb-aa3a-d91a4d27ba8b-0', tool_calls=[{'name': 'search_tavily', 'args': {'query': 'San Francisco weather'}, 'id': 'call_VMvCRxAUxacA29DKlpwSdmYr', 'type': 'tool_call'}], usage_meta